#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
print(client.list_database_names())

['admin', 'config', 'exercices', 'local', 'peaky', 'series']


In [35]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [6]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

## Question 0

### Netoyer les données

In [7]:
print(df_ks["deadline"][df_ks["deadline"].apply(len) != 10])

66141    USD
Name: deadline, dtype: object


In [8]:
print(df_ks["launched"][df_ks["launched"].apply(len) != 19])

66141    700.0
Name: launched, dtype: object


In [36]:
# On voit une valeur non cohérentes on va donc la supprimer
df_ks.drop(66141,inplace=True)

In [14]:
df_ks.query("deadline == 'USD'")

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real


In [112]:
def convertToISOFORMAT(date):
    import dateutil.parser as parser
    d2 = parser.parse(date)
    return d2

In [79]:
# Il faut convertir les dates au format unix pour pouvoir réaliser des opérations dessus
df_ks["deadline"] = df_ks["deadline"].apply(convertToISOFORMAT)


In [80]:
df_ks["launched"] = df_ks["launched"].apply(convertToISOFORMAT)

In [81]:
df_ks_dict = df_ks.to_dict('index')

### Importer les données

In [102]:
DOCUMENTS = list(df_ks_dict.values())

In [103]:
collection.insert_many(DOCUMENTS)

## Question 1  

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.


In [111]:
list(collection.find().sort([("usd_pledged_real", -1)]).limit(5))

[{'_id': ObjectId('61b1bed36524ea333410350c'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': datetime.datetime(2014, 8, 30, 0, 0),
  'goal': '50000.0',
  'launched': datetime.datetime(2014, 7, 8, 10, 14, 37),
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('61b1bed36524ea3334109005'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': datetime.datetime(2016, 6, 30, 0, 0),
  'goal': '1000000.0',
  'launched': datetime.datetime(2016, 5, 24, 15, 49, 52),
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 127798

## Question 2

- 2) Compter le nombre de projets ayant atteint leur but.


In [121]:
# Toutes les categories
cur = collection.aggregate([{"$group" : {"_id" : "$state", "Nombre" : {"$sum" : 1}}}])
list(cur)
#53040 ont atteint leur but

[{'_id': 'live', 'Nombre': 1133},
 {'_id': 'successful', 'Nombre': 53040},
 {'_id': 'canceled', 'Nombre': 15359},
 {'_id': 'undefined', 'Nombre': 1395},
 {'_id': 'failed', 'Nombre': 78334},
 {'_id': 'suspended', 'Nombre': 738}]

In [122]:
len(list(
    collection.find({"state":"successful"})
))

53040

## Question 3

- 3) Compter le nombre de projets pour chaque catégorie.


In [120]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "Nombre" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Comics', 'Nombre': 1931},
 {'_id': 'Fabrication Tools', 'Nombre': 96},
 {'_id': 'Apparel', 'Nombre': 2827},
 {'_id': 'People', 'Nombre': 440},
 {'_id': 'Music', 'Nombre': 6229},
 {'_id': 'Crafts', 'Nombre': 1834},
 {'_id': 'Woodworking', 'Nombre': 433},
 {'_id': 'Spaces', 'Nombre': 331},
 {'_id': 'Cookbooks', 'Nombre': 217},
 {'_id': 'Music Videos', 'Nombre': 299},
 {'_id': 'Fashion', 'Nombre': 3379},
 {'_id': 'Print', 'Nombre': 288},
 {'_id': 'Hardware', 'Nombre': 1431},
 {'_id': 'Nonfiction', 'Nombre': 3390},
 {'_id': 'Art Books', 'Nombre': 1065},
 {'_id': 'Playing Cards', 'Nombre': 963},
 {'_id': 'Civic Design', 'Nombre': 130},
 {'_id': 'Electronic Music', 'Nombre': 858},
 {'_id': 'Tabletop Games', 'Nombre': 5581},
 {'_id': 'Camera Equipment', 'Nombre': 165},
 {'_id': 'Vegan', 'Nombre': 229},
 {'_id': 'Painting', 'Nombre': 1288},
 {'_id': 'Residencies', 'Nombre': 32},
 {'_id': 'Science Fiction', 'Nombre': 274},
 {'_id': 'Knitting', 'Nombre': 78},
 {'_id': 'Crochet', 'Nombr

## Question 4

- 4) Compter le nombre de projets français ayant été instanciés avant 2016.


In [104]:
d2 = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')


In [108]:
list(collection.find({
    "deadline" : {"$lt": d2},
    "country" : "FR"
}))

[{'_id': ObjectId('61b1bed26524ea33340f0c24'),
  'ID': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'currency': 'EUR',
  'deadline': datetime.datetime(2015, 7, 1, 0, 0),
  'goal': 20000.0,
  'launched': datetime.datetime(2015, 5, 27, 11, 29, 28),
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': ObjectId('61b1bed26524ea33340f0dce'),
  'ID': 400288569,
  'name': 'Dracucat : A vampire kitten card game',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'EUR',
  'deadline': datetime.datetime(2015, 10, 29, 0, 0),
  'goal': 1900.0,
  'launched': datetime.datetime(2015, 9, 29, 7, 4, 8),
  'pledged': 2944.0,
  'state': 'successful',
  'backers': 129,
  'country': 'FR',
  'usd pledged': 3295.37237632,
  'usd_pledged_real': 3243.3980764357875},
 {'_id': ObjectId('61b1bed26524ea33340f0f8e'),
  'ID': 1990332786

## Question 5

- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [123]:
list(collection.find({
    "usd_pledged_real" : {"$gt": 200000},
    "country" : "US"
}))

[{'_id': ObjectId('61b1bed26524ea33340f0a17'),
  'ID': 217543389,
  'name': 'The uKeg Pressurized Growler for Fresh Beer',
  'category': 'Drinks',
  'main_category': 'Food',
  'currency': 'USD',
  'deadline': datetime.datetime(2014, 12, 8, 0, 0),
  'goal': 75000.0,
  'launched': datetime.datetime(2014, 10, 15, 6, 34, 48),
  'pledged': 1559525.68,
  'state': 'successful',
  'backers': 10293,
  'country': 'US',
  'usd pledged': 1559525.68,
  'usd_pledged_real': 1559525.68},
 {'_id': ObjectId('61b1bed26524ea33340f0aa7'),
  'ID': 909248984,
  'name': 'Redux COURG - Hybrid Watches with Missions to Tackle',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': datetime.datetime(2015, 8, 20, 0, 0),
  'goal': 30000.0,
  'launched': datetime.datetime(2015, 7, 21, 19, 1, 41),
  'pledged': 692912.0,
  'state': 'successful',
  'backers': 2200,
  'country': 'US',
  'usd pledged': 692912.0,
  'usd_pledged_real': 692912.0},
 {'_id': ObjectId('61b1bed26524ea33

## Question 6 

- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [128]:
collection.create_index([("name",  "text")])


'name_text'

In [129]:
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [127]:
cur = collection.find( { "$text": { "$search": "Sport" } } )
list(cur)

[{'_id': ObjectId('61b1bed26524ea33340fb93d'),
  'ID': 1875366029,
  'name': 'Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!',
  'category': 'Webseries',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': datetime.datetime(2011, 1, 15, 0, 0),
  'goal': 4500.0,
  'launched': datetime.datetime(2010, 12, 16, 8, 40, 2),
  'pledged': 25.0,
  'state': 'failed',
  'backers': 1,
  'country': 'US',
  'usd pledged': 25.0,
  'usd_pledged_real': 25.0},
 {'_id': ObjectId('61b1bed26524ea33340f3b1f'),
  'ID': 1126822169,
  'name': 'Frey Sports App - We connect sports people.',
  'category': 'Apps',
  'main_category': 'Technology',
  'currency': 'DKK',
  'deadline': datetime.datetime(2017, 1, 11, 0, 0),
  'goal': 25000.0,
  'launched': datetime.datetime(2016, 12, 12, 11, 30, 28),
  'pledged': 0.0,
  'state': 'failed',
  'backers': 0,
  'country': 'DK',
  'usd pledged': 0.0,
  'usd_pledged_real': 0.0},
 {'_id': ObjectId('61b1bed36524ea333411219d'),
  'ID': 1081541783,
  '